In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import cv2
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
import json
data_dir = '/Users/jayanthrachamadugu/Downloads/fruits'
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
print(f"Training data: {train_generator.samples} images in {len(train_generator.class_indices)} classes")
print(f"Validation data: {validation_generator.samples} images in {len(validation_generator.class_indices)} classes")
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
try:
    model.fit(train_generator, validation_data=validation_generator, epochs=10)
except Exception as e:
    print(f"Error during model training: {e}")
model.save('/Users/jayanthrachamadugu/Downloads/fruit_model.keras')
import json
with open('/Users/jayanthrachamadugu/Downloads/class_labels.json', 'w') as f:
    json.dump(train_generator.class_indices, f)
model = tf.keras.models.load_model('/Users/jayanthrachamadugu/Downloads/fruit_model.keras')
with open('/Users/jayanthrachamadugu/Downloads/class_labels.json', 'r') as f:
    class_indices = json.load(f)
class_labels = {v: k for k, v in class_indices.items()}
def predict_fruit(img_path):
    # Load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    predictions = model.predict(img_array)
    print("Raw predictions:", predictions)  # Print raw predictions
    predicted_class = np.argmax(predictions, axis=1)[0]
    if predicted_class not in class_labels:
        raise ValueError(f"Predicted class index {predicted_class} is not in class_labels.")
    
    fruit_name = class_labels[predicted_class]
    print("Predicted class index:", predicted_class)
    print("Predicted fruit name:", fruit_name)
    return fruit_name
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            fruit_name = predict_fruit(file_path)
            result_label.config(text=f"Predicted Fruit: {fruit_name}")
        except Exception as e:
            result_label.config(text=f"Error: {str(e)}")

root = tk.Tk()
root.title("Fruit Recognition")

upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=20)

result_label = tk.Label(root, text="Predicted Fruit: ")
result_label.pack(pady=20)

root.mainloop()


Found 6 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Training data: 6 images in 3 classes
Validation data: 0 images in 3 classes
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3333 - loss: 1.2909Error during model training: Must provide at least one structure
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
Raw predictions: [[0.00531465 0.9929796  0.00170578]]
Predicted class index: 1
Predicted fruit name: banana
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Raw predictions: [[0.01008673 0.00248513 0.9874281 ]]
Predicted class index: 2
Predicted fruit name: orange
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Raw predictions: [[0.77531207 0.10106367 0.12362432]]
Predicted class index: 0
Predicted fruit name: apple
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Raw predictions: [[0.01008673 0.00248513 0.9874281 ]]
Predicted class index: 2
Predicted fruit name: orange
